In [1]:
import pandas as pd
import numpy as np
import skmem

train_set = pd.read_csv('../data/train.csv')
test_set = pd.read_csv('../data/test.csv')

weather_train_set = pd.read_csv('../data/weather_train.csv')
weather_test_set = pd.read_csv('../data/weather_test.csv')

building_metadata = pd.read_csv('../data/building_metadata.csv')

sample_submission = pd.read_csv('../data/sample_submission.csv')

In [2]:
train_data = pd.merge(train_set, building_metadata, on='building_id')
train_data = pd.merge(train_data, weather_train_set, on=['site_id', 'timestamp'])

In [3]:
mr = skmem.MemReducer()
train_data = mr.fit_transform(train_data)

Getting memory usage.
Memory in: 5092.41 MB
Starting integers.
Downcast 4 standard integer columns.
Starting floats.
auto
Skipping columns that are not np.float64
Downcast 10 float columns.
Starting objects.
Converted 2 columns to categories.
Memory out: 1133.29 MB Reduction: 77.7%



In [4]:
train_data['primary_use'] = pd.Categorical(train_data['primary_use']).codes

In [5]:
test_data = pd.merge(test_set, building_metadata, on='building_id')
test_data = pd.merge(test_data, weather_test_set, on=['site_id', 'timestamp'])

In [6]:
mr = skmem.MemReducer()
test_data = mr.fit_transform(test_data)

Getting memory usage.
Memory in: 10502.28 MB
Starting integers.
Downcast 5 standard integer columns.
Starting floats.
auto
Skipping columns that are not np.float64
Downcast 9 float columns.
Starting objects.
Converted 2 columns to categories.
Memory out: 2336.77 MB Reduction: 77.7%



In [7]:
test_data['primary_use'] = pd.Categorical(test_data['primary_use']).codes

In [36]:
train_data.isnull().sum() / len(test_data) * 100

building_id            0.000000
meter                  0.000000
timestamp              0.000000
meter_reading          0.000000
site_id                0.000000
primary_use            0.000000
square_feet            0.000000
air_temperature        0.014851
cloud_coverage        21.048604
dew_temperature        0.023242
precip_depth_1_hr      8.816034
sea_level_pressure     2.749912
wind_direction         3.273734
wind_speed             0.128151
dtype: float64

In [9]:
test_data.isnull().sum() / len(test_data) * 100

row_id                 0.000000
building_id            0.000000
meter                  0.000000
timestamp              0.000000
site_id                0.000000
primary_use            0.000000
square_feet            0.000000
year_built            59.148082
floor_count           82.724073
air_temperature        0.055115
cloud_coverage        46.611607
dew_temperature        0.148848
precip_depth_1_hr     18.319990
sea_level_pressure     5.585245
wind_direction         6.698144
wind_speed             0.248346
dtype: float64

In [44]:
train_data.drop(columns=['year_built', 'floor_count'], inplace=True)
test_data.drop(columns=['year_built', 'floor_count'], inplace=True)

In [37]:
na_features = ['air_temperature', 'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr', 'sea_level_pressure', 'wind_direction', 'wind_speed']
for feature in na_features:
    train_data[feature] = train_data[feature].fillna(train_data[feature].mean())
    test_data[feature] = test_data[feature].fillna(test_data[feature].mean())

In [41]:
train_data.isnull().sum()

building_id           0
meter                 0
timestamp             0
meter_reading         0
site_id               0
primary_use           0
square_feet           0
air_temperature       0
cloud_coverage        0
dew_temperature       0
precip_depth_1_hr     0
sea_level_pressure    0
wind_direction        0
wind_speed            0
dtype: int64

In [45]:
test_data.isnull().sum()

row_id                0
building_id           0
meter                 0
timestamp             0
site_id               0
primary_use           0
square_feet           0
air_temperature       0
cloud_coverage        0
dew_temperature       0
precip_depth_1_hr     0
sea_level_pressure    0
wind_direction        0
wind_speed            0
dtype: int64

In [51]:
train_data['timestamp'] = pd.to_datetime(train_data['timestamp'])

In [74]:
train_data['hour'] = train_data['timestamp'].dt.hour
train_data['day'] = train_data['timestamp'].dt.day
train_data['week'] = train_data['timestamp'].dt.isocalendar().week
train_data['month'] = train_data['timestamp'].dt.month
train_data['year'] = train_data['timestamp'].dt.year
train_data.drop(columns='timestamp', inplace=True)
print(train_data)

          building_id  meter  meter_reading  site_id  primary_use  \
0                   0      0       0.000000        0            0   
1                   1      0       0.000000        0            0   
2                   2      0       0.000000        0            0   
3                   3      0       0.000000        0            0   
4                   4      0       0.000000        0            0   
...               ...    ...            ...      ...          ...   
20125600         1400      1      15.375300       15            4   
20125601         1400      1      25.084801       15            4   
20125602         1400      1      32.343899       15            4   
20125603         1400      1      24.221399       15            4   
20125604         1400      1      16.355301       15            4   

          square_feet  air_temperature  cloud_coverage  dew_temperature  \
0                7432             25.0        6.000000             20.0   
1                2720

In [75]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20125605 entries, 0 to 20125604
Data columns (total 18 columns):
 #   Column              Dtype  
---  ------              -----  
 0   building_id         uint16 
 1   meter               uint8  
 2   meter_reading       float32
 3   site_id             uint8  
 4   primary_use         int8   
 5   square_feet         uint32 
 6   air_temperature     float32
 7   cloud_coverage      float32
 8   dew_temperature     float32
 9   precip_depth_1_hr   float32
 10  sea_level_pressure  float32
 11  wind_direction      float32
 12  wind_speed          float32
 13  hour                int64  
 14  day                 int64  
 15  week                UInt32 
 16  month               int64  
 17  year                int64  
dtypes: UInt32(1), float32(8), int64(4), int8(1), uint16(1), uint32(1), uint8(2)
memory usage: 1.6 GB
